In [ ]:
# download  model-repo 

from hf_models_download import HFModelsDownloader_SnapShot 


model_cache_root = r"E:\local_models\huggingface\cache\hub"
model_name = r"Qwen/Qwen3-1.7B"
target_dir = r"E:\local_models\huggingface\local\qwen3-1.7b"

snapshot_load = HFModelsDownloader_SnapShot(cache_dir=model_cache_root)
snapshot_load.download_repo(
    repo_type="model",
    repo_id=model_name,
    local_dir=target_dir,
)


###  上下文检索

- 上下文提取  太占用内存了，

In [1]:
# markdown    _filtered.md : test_str   _filtered.json: text_meta 

from llama_index.llms.huggingface import HuggingFaceLLM 
from llama_index.core import Settings 
chat_model = HuggingFaceLLM(
    # cache_root可以省，model_name
    # or  local_dir
    model_name     = r"Qwen/Qwen3-1.7B",
    tokenizer_name = r"Qwen/Qwen3-1.7B",
    context_window = 3900,  
    max_new_tokens = 640,
    generate_kwargs={"temperature": 0.7, "top_k": 30, "top_p": 0.95},
    device_map     ='cpu' 
)
# settings llm 
Settings.llm = chat_model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
# embedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embedding = HuggingFaceEmbedding(
    # cache
    model_name="Qwen/Qwen3-Embedding-0.6B",
    cache_folder="E:/local_models/huggingface/cache/hub", 
    # local-dir
    # model_name = r"E:\local_models\huggingface\local\path_to_qwen3Embedding0.6b_load_dir
    max_length=1024,
    trust_remote_code=True,
    model_kwargs={"local_files_only": True},   # 允许联网 False , 禁止联网 True
    device='cpu',
)

from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.storage.docstore.simple_docstore import (
    SimpleDocumentStore,
)

# initialize documen store 
docstore = SimpleDocumentStore()

# create storage contexts
storage_context = StorageContext.from_defaults(docstore=docstore)
storage_context_no_extra_context = StorageContext.from_defaults()
text_splitter = TokenTextSplitter(
    separator=" ", chunk_size=600, chunk_overlap=200
)

In [4]:
# from llama_index.core.extractors import DocumentContextExtractor
# # 上下文提取器件   太占用内存了--
# context_extractor = DocumentContextExtractor(
#     docstore=docstore,                # 必填。用于拿到“整篇文档/原文”，以便从中截取上下文供 LLM 概括
#     max_context_length=1000,          # 必填。喂给 LLM 的“源上下文”最大长度（以 token/字符为单位，依版本而定）
#     llm=Settings.llm,                 # 选填。默认用 Settings.llm；你也可传一个更轻的小模型专门做摘要
#     oversized_document_strategy="warn",  # 超过 max_context_length 的处理：常见有 "warn" / "truncate" / "error"
#     max_output_tokens=400,            # 生成的“上下文摘要”最大长度
#     key="context",                    # 写入 node.metadata 的键名
#     prompt=DocumentContextExtractor.SUCCINCT_CONTEXT_PROMPT,  # 可自定义提示词
# )


# - docstore：必须。它让提取器能从文档 ID 找回完整原文或较大的片段，再按 max_context_length 截取，喂给 LLM 生成摘要。
# - max_context_length：控制输入给 LLM 的源文本长度。太大 → 注意力是 O(L²) → CPU/GPU 内存暴涨。一般建议 1k~2k。
# - oversized_document_strategy：
#     - "warn"：logging 
#     - "ignore"： pass 
#     - "error"：直接抛错，强制你缩短或调配置。
# - llm：可以单独换成更轻的模型（如 1B 左右）专门做摘要，主问答/检索仍用你的大模型。
# - max_output_tokens：控制输出摘要长度，避免写入的 metadata 过肥。
# - key / prompt：控制写到哪、用什么提示词。

# - 放置顺序：一般 transformations=[text_splitter, context_extractor]，先切块再为每块生成上下文摘要。


'  \n- docstore：必须。它让提取器能从文档 ID 找回完整原文或较大的片段，再按 max_context_length 截取，喂给 LLM 生成摘要。\n- max_context_length：控制输入给 LLM 的源文本长度。太大 → 注意力是 O(L²) → CPU/GPU 内存暴涨。一般建议 1k~2k。\n- oversized_document_strategy：\n    - "warn"：超长时仅告警但仍尝试处理（风险是 OOM）；\n    - "truncate"：推荐，直接把源文本截断到 max_context_length 再生成；\n    - "error"：直接抛错，强制你缩短或调配置。\n- llm：可以单独换成更轻的模型（如 1B 左右）专门做摘要，主问答/检索仍用你的大模型。\n- max_output_tokens：控制输出摘要长度，避免写入的 metadata 过肥。\n- key / prompt：控制写到哪、用什么提示词。\n\n- 放置顺序：一般 transformations=[text_splitter, context_extractor]，先切块再为每块生成上下文摘要。\n'

In [1]:

from pathlib import Path 

# 
from llama_index.readers.file.markdown import MarkdownReader

from llama_index.core import SimpleDirectoryReader 

data_root = r"D:\ddesktop\agentdemos\codespace\zhuanliParser\result"
in_files = str(next(Path(data_root).glob("*/full_filtered.md"), None))
print(in_files)
# in_files = [str(x) for x in in_files]
# assert len(in_files) == 14, "in_files  counts  are not 14 ?"
# assert len([in_files]) == 1, "in_files  counts  are not 1 ?"

reader = SimpleDirectoryReader(input_files=[in_files])
documents = reader.load_data()

2025-09-03 17:55:18,919 - INFO - Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
2025-09-03 17:55:18,920 - INFO - NumExpr defaulting to 16 threads.


D:\ddesktop\agentdemos\codespace\zhuanliParser\result\CN201721328994.5-一种机器人足端结构.pdf-e856a75f-7fe7-4813-a860-05ca62419ac7\full_filtered.md


In [4]:
import nest_asyncio 

nest_asyncio.apply()

# need to add documents directly for the DocumentContextExtractor to work
storage_context.docstore.add_documents(documents)
index = VectorStoreIndex.from_documents(
    documents=documents,
    storage_context=storage_context,
    embed_model=embedding,
    transformations=[text_splitter],   # 
)

index_nocontext = VectorStoreIndex.from_documents(
    documents=documents,
    storage_context=storage_context_no_extra_context,
    embed_model=embedding,
    transformations=[text_splitter],
)


In [7]:
test_question = "测控单元是什么样的?"
retriever = index.as_retriever(similarity_top_k=2)
nodes_fromcontext = retriever.retrieve(test_question)

retriever_nocontext = index_nocontext.as_retriever(similarity_top_k=2)
nodes_nocontext = retriever_nocontext.retrieve(test_question)

In [8]:
# Print each node's content
print("==========")
print("NO CONTEXT")
for i, node in enumerate(nodes_nocontext, 1):
    print(f"\nChunk {i}:")
    print(f"Score: {node.score}")  # Similarity score
    print(f"Content: {node.node.text}")  # The actual text content

# Print each node's content
print("==========")
print("WITH CONTEXT")   # 没办法对比，提取上下文跑不动
for i, node in enumerate(nodes_fromcontext, 1):
    print(f"\nChunk {i}:")
    print(f"Score: {node.score}")  # Similarity score
    print(f"Content: {node.node.text}")  # The actual text content

NO CONTEXT

Chunk 1:
Score: 0.4905331773059238
Content: 所述敏感部2- 1- 1至少设有一对相互平行的平面区域2- 1- 1- 1，所述相互平行的平面区域2- 1- 1- 1两侧粘接有至少一组用于感应形变的应变片2- 2。所述应变片2- 2把支撑力对敏感部2- 1- 1产生的形变转化为自身电阻的改变量，应变片2- 2配合测控单元2- 4，产生与外部支撑力大小相对应的微弱电信号，测控单元2- 4上的电信号放大芯片采集此微弱电信号并进行放大和处理，并通过数字通讯接口传递给机器人主控制板。

[0035] 所述测控单元2- 4安装于传感部件2- 1的弯折空间内, 所述测控单元2- 4设有电信号放大芯片和三轴加速度传感器, 或设有三轴陀螺仪传感器, 或设有一片微控制器MCU。三轴加速度芯片, 用于检测足端的三轴加速度, 通过三轴加速度数据或再附加配合三轴陀螺仪传感器可以计算获得足的姿态、足端碰撞、足端打滑等情况。三轴陀螺仪传感器, 用于检测足端三轴的角速度, 可以配合三轴加速度计进一步完善足端姿态的解算, 或者单纯测量足端及其相连机器人腿部连杆的角速度。微控制器MCU, 控制测控单元2-

Chunk 2:
Score: 0.46583514996833364
Content: 3、测控单元2- 4、足基座2- 5。足基座2- 5为横放的圆柱结构，所述圆柱结构的柱面设有矩形截面槽，所述传感部件2- 1位于足基座2- 5的固定槽内，通过粘接或者螺钉连接的方式固定。所述足垫3通过粘接或者熔融的方式连接到足基座2- 5上。传感部件2- 1和足基座2- 5也可以是一个单独的整体零件。[0032] 所述传感部件2- 1形状近似于一个倾斜一定角度的字母“Z”，所述传感部件2- 1“Z”字的中部分是所述用于测力能产生较大变形的敏感部2- 1- 1，传感部件2- 1由高屈服强度材料制成。所述高屈服强度材料为铝合金或碳钢或钛合金。

[0033] 足基座总成2的外部设置一保护罩总成1，保护罩总成1通过螺钉连接或者粘接的方式固定在足基座总成2上，将足基座总成2内部的传感部件2- 1、应变片2- 2、测控单元2- 4以及足基座2- 5与传感部件2-
WITH CONTEXT

Chunk 1:
Score: 0.49053317730592

### 元数据

- 首先，meta_data已提取到  MetaDict_filtered.json 中
- llama-index 内置的node机制初步 load一下markdown，node.metadata
- node.metadata  |  MetaDict_filtered.json   --update node.metadata 

In [ ]:
# 版本问题  



In [3]:

# parser   extractor 
import span_marker
from llama_index.extractors.entity import EntityExtractor 
from llama_index.core.node_parser import SentenceSplitter 


entity_extractor = EntityExtractor(
    prediction_threshold=0.5,
    label_entities=False,
    device='cpu',
)

node_parser = SentenceSplitter()

transformations = [node_parser, entity_extractor]

# 当你把 transformations 设为 [node_parser, entity_extractor] 时，流水线大致是：
# 1. SentenceSplitter 先跑：把每个 Document 切成多个 Node
# （每个 Node 有 text、metadata 等）。
# 2. EntityExtractor 后跑：对每个 Node 的文本跑一次 NER，
#  把抽到的实体写进该 Node 的 metadata（通常是 metadata["entities"]）。
# 3. 后续建索引：VectorStoreIndex.from_documents(..., transformations=[...])
# 会用完成增强后的 Nodes（包含 entities 等元数据）来做嵌入+入库。
# 这样你可以在检索时结合元数据过滤（比如只要含某公司名/人名的块）。
# 这类“元数据增强 → 过滤检索”的模式是官方推荐的实践之一

# load data 
# documents 

# extract metadata 
from llama_index.core.ingestion import IngestionPipeline 

pipeline = IngestionPipeline(transformations=transformations)

nodes = pipeline.run(documents)

import random 
samples = random.sample(nodes, 5)
for node in samples:
    print(node.metadata)


RuntimeError: Failed to import transformers.models.auto.modeling_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
DLL load failed while importing lib: 找不到指定的程序。